# Save land-only XCO2 data
---

In [1]:
import sys
sys.path.insert(0, "../src")

In [2]:
import numpy as np
import pandas as pd
import xarray
import regionmask

import matplotlib.pyplot as plt
import cartopy.crs as ccrs

In [4]:
DS_xco2 = xarray.open_dataset("../data/exp_pro/OCO2_Lite_XCO2.nc")
DS_xco2

<xarray.Dataset>
Dimensions:   (time: 194231572)
Coordinates:
    lon       (time) float32 ...
    lat       (time) float32 ...
  * time      (time) datetime64[ns] 2014-09-06T02:07:25.228014848 ... 2020-11...
Data variables:
    xco2      (time) float32 ...
    xco2_var  (time) float32 ...

In [5]:
# Relabel lon-lat coords on 1-degree

# Establish grid
lon_bins = np.arange(-180, 180)
lat_bins = np.arange(-90, 90)
lon_centers = (lon_bins[1:] + lon_bins[:-1]) / 2
lat_centers = (lat_bins[1:] + lat_bins[:-1]) / 2

# Assign grid labels
df_grid_xco2 = DS_xco2.to_dataframe().reset_index()
df_grid_xco2["lon_centers"] = pd.cut(df_grid_xco2.lon, lon_bins, labels=lon_centers).astype(float)
df_grid_xco2["lat_centers"] = pd.cut(df_grid_xco2.lat, lat_bins, labels=lat_centers).astype(float)

df_grid_xco2

,time,xco2,xco2_var,lon,lat,lon_centers,lat_centers
0,2014-09-06 02:07:25.228014848,394.484650,1.235173,160.511780,54.465744,160.5,54.5
1,2014-09-06 02:07:25.561990144,392.932068,1.228083,160.501877,54.485321,160.5,54.5
2,2014-09-06 02:07:25.615017472,394.069733,1.165426,160.537964,54.482136,160.5,54.5
3,2014-09-06 02:07:25.920990464,394.211243,1.244987,160.510025,54.503334,160.5,54.5
4,2014-09-06 02:07:25.947986944,394.436035,1.214056,160.528091,54.501705,160.5,54.5
...,...,...,...,...,...,...,...
194231567,2020-11-30 23:44:35.749008384,414.641174,0.874593,-159.673721,28.139866,-159.5,28.5
194231568,2020-11-30 23:44:36.083023872,414.321808,0.870935,-159.679718,28.157648,-159.5,28.5
194231569,2020-11-30 23:46:35.228022272,413.639832,1.178670,-161.990387,34.649147,-161.5,34.5
194231570,2020-11-30 23:46:35.587988224,413.656372,1.042477,-161.986465,34.653782,-161.5,34.5


In [6]:
# Construct ocean mask
land = regionmask.defined_regions.natural_earth.land_110
mask = land.mask(lon_centers, lat_centers)
# mask.plot(add_colorbar=False);

df_mask = mask.to_dataframe().reset_index().rename(columns={"lon": "lon_centers", "lat": "lat_centers"})
df_mask

,lat_centers,lon_centers,region
0,-89.5,-179.5,0.0
1,-89.5,-178.5,0.0
2,-89.5,-177.5,0.0
3,-89.5,-176.5,0.0
4,-89.5,-175.5,0.0
...,...,...,...
64256,88.5,174.5,NaN
64257,88.5,175.5,NaN
64258,88.5,176.5,NaN
64259,88.5,177.5,NaN


In [7]:
mask

<xarray.DataArray 'region' (lat: 179, lon: 359)>
array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])
Coordinates:
  * lat      (lat) float64 -89.5 -88.5 -87.5 -86.5 -85.5 ... 85.5 86.5 87.5 88.5
  * lon      (lon) float64 -179.5 -178.5 -177.5 -176.5 ... 176.5 177.5 178.5

In [8]:
# Merge land mask, drop ocean values, convert to dataset

DS_temp = (
    pd.merge(df_grid_xco2, df_mask, on=["lon_centers", "lat_centers"], how="left")
    .dropna(subset=["region"])
    .drop(columns=["lon_centers", "lat_centers", "region"])
    .set_index(["time"])
    .to_xarray()
)

DS_xco2_land = xarray.Dataset(
    {
        "xco2": (["time"], DS_temp.xco2),
        "xco2_var": (["time"], DS_temp.xco2_var),
    },
    coords={
        "lon": (["time"], DS_temp.lon),
        "lat": (["time"], DS_temp.lat),
        "time": DS_temp.time,
    },
)

DS_xco2_land

<xarray.Dataset>
Dimensions:   (time: 62942059)
Coordinates:
    lon       (time) float32 160.51178 160.50188 ... -111.281555 -111.26298
    lat       (time) float32 54.465744 54.48532 ... 43.457752 43.443253
  * time      (time) datetime64[ns] 2014-09-06T02:07:25.228014848 ... 2020-11...
Data variables:
    xco2      (time) float32 394.48465 392.93207 ... 417.20496 416.9326
    xco2_var  (time) float32 1.2351726 1.2280829 ... 1.2599125 1.4953663

In [9]:
DS_xco2_land.to_netcdf("../data/exp_pro/OCO2_Lite_XCO2_land.nc", format="NETCDF4")